### decompose


- 5.24早修改table format 空格\xa
- 5.24 下午修改str_normalize 
- 5.24 晚 72需要复查
- 5.25 早list query strip() 
- 5.27 修改total，尝试disambiguous/  改成PIPE

In [1]:
import sys
sys.path.append('/media/disk1/chatgpt/zh/tabular_data')

In [2]:
import pandas as pd
from utils import parse_specific_composition, add_row_number, parse_specific_composition_zh
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from data_loader import TableFormat, TableLoader
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from sqlalchemy import create_engine
from executor import SQLManager
import sqlparse
embeddings = HuggingFaceBgeEmbeddings(
            model_name='BAAI/bge-large-en',
            model_kwargs={'device': 'cuda:2', 'trust_remote_code': True},
            encode_kwargs={'normalize_embeddings': True})

In [3]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
query_examples = [
                  "what was the time difference between the first place finisher and the eighth place finisher?",
                  "other than william stuart price, which other businessman was born in tulsa?",
                  "which canadian city had the most passengers traveling from manzanillo international airport in 2013?"
                # "what is the next most populous district after haridwar?",(70)
                  ]
new_query_examples = [
                      "what was the time for the first place finisher?; what was the time for the eighth place finisher?",
                      "was william stuart price born in tulsa?; who was born in tulsa?",
                      "how many passengers do each airline from canadian city have?; which canadian city had the most passengers?"
                    # "what are the districts after haridwar?; what is the next most populous district after haridwar?",
                    #   "When did polona hercog partner with alberta brianti?; When did polona hercog partner with stephanie vogt?",
                      ]
num_k = 3
inds = [1, 11, 86, 70, 42]
table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
normalised_data = [table_loader.normalize_table(table_loader.dataset[inds[i]]) for i in range(num_k)]
example_samples = [TableFormat(format='none', data=normalised_data[i], save_embedding=True, embeddings=embeddings).get_sample_data(sample_type='embedding', query=normalised_data[i]['query']) for i in range(num_k)]
examples = [TableFormat.format_PIPE(example_samples[i], normalised_data[i]['table']['caption']) for i in range(num_k)]

examples_prompt = PromptTemplate(input_variables=["query", "table", "new_query"], template=
"""Sub-Table: {table}
Query: {query}
Decompose query: {new_query}""")

examples_dict = [{"query": query_examples[i],
                                    "table": examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
decompose_prompt_wiki = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are capable of converting complex query into sub queries. Based on the table, decompose original query into at most 2 complete sub queries which can solve original query. Output new query directly.""",
    suffix=
    """Sub-Table: {table}
Query: {query}
Decompose query: """,
    input_variables=["query", "table"],
)



In [4]:
task_name = 'wikitable'
split = 'test'
model_name = 'gpt-3.5-turbo-0125'
# model = ChatOpenAI(model_name=model_name, openai_api_base="https://api.chatanywhere.com.cn/v1",
#                        openai_api_key="sk-WZtqZEeuE0Xb6syVghDgAxdwe0ASWLkQRGxl61UI7B9RqNC4", temperature=0.7).bind(logprobs=True)
schema_information = pd.read_csv(f"../result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
aug_information = pd.read_csv(f"../result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
composition_information = pd.read_csv(f"../result/aug/{task_name}_{split}_composition.csv", index_col='table_id')
engine = create_engine('sqlite:////media/disk1/chatgpt/zh/tabular_data/db/sqlite/wiki_query.db', echo=False)
manager = SQLManager(engine=engine)

In [5]:
table_loader = TableLoader(table_name=task_name, split='test', use_sample=False, small_test=False)
sample = table_loader.normalize_table(table_loader.dataset[2])

### disambiguous

In [6]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
inds = [5, 11, 46]
num_k = 3
table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
normalised_data = [table_loader.normalize_table(table_loader.dataset[inds[i]]) for i in range(num_k)]
example_samples = [TableFormat(format='none', data=normalised_data[i], save_embedding=True,embeddings=embeddings).get_sample_data(sample_type='embedding', query=normalised_data[i]['query']) for i in range(num_k)]
examples = [TableFormat.format_PIPE(example_samples[i], normalised_data[i]['table']['caption']) for i in range(num_k)]
new_query_examples = ["was the Sandia Peak Tramway before or after the 3S Aerial Tramway in terms of Year_of_inauguration?",
                      "other than William Stuart Price, which other businessman's was in Tulsa in terms of Hometown?",
                      "How many players are G\\nF in terms of Position?"]
examples_prompt = PromptTemplate(input_variables=["query", "new_query"], template=
"""
Query: {query}
Table: {table}
New query: {new_query}""")

examples_dict = [{"query": table_loader.dataset[inds[i]]['question'],
                  "table": examples[i],
                    "new_query": new_query_examples[i]} for i in range(num_k)]
disambiguous_prompt_wiki = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    # prefix="""Based on the given table, your task is to rewrite the query to resolve ambiguity and ensure the question is consistent with the table. 
    # This requires pinpointing elements of the question to table contents and rewriting the question to ensure a consistent, clear interpretation. """,
    suffix=
    """
Query: {query}
Table: {table}
New query: """,
    input_variables=["query", "table"],
)

### step-back

In [7]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
inds = [11, 182]
num_k = 2
table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
normalised_data = [table_loader.normalize_table(table_loader.dataset[inds[i]]) for i in range(num_k)]
example_samples = [TableFormat(format='none', data=normalised_data[i], save_embedding=True,embeddings=embeddings).get_sample_data(sample_type='embedding', query=normalised_data[i]['query']) for i in range(num_k)]
examples = [TableFormat.format_PIPE(example_samples[i], normalised_data[i]['table']['caption']) for i in range(num_k)]
new_query_examples = [
    # "Which country uses the US dollar as its currency and has the Federal Reserve as its central bank?",
    "which business man was born in tulsa?",
    "what is the network owned by national polytechnic institute?"
    ]
examples_prompt = PromptTemplate(input_variables=["query", "new_query"], template=
"""
Query: {query}
Table: {table}
New query: {new_query}""")

examples_dict = [{"query": table_loader.dataset[inds[i]]['question'],
                  "table": examples[i],
                    "new_query": new_query_examples[i]} for i in range(num_k)]
step_back_prompt_wiki = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""Based on the table, your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.""",
    suffix=
    """
Query: {query}
Table: {table}
New query: """,
    input_variables=["query", "table"],
)

In [8]:
def get_k_shot_with_answer(k: int=1):
    sqls = ["SELECT MAX(winners_c_from_previous_round) FROM DF;"
            ]
    thoughts = ["Based on the the SQL query provided, The sub-table shows that the highest number of winner from a previous round was 54."]
    tables = ["""<table>
<thead>
<tr><th>  MAX(winners_c_from_previous_round)</th></tr>
</thead>
<tbody>
<tr><td>54                             </td></tr>
</tbody>
</table>"""]
    tables_pipe = ["""/*
table caption : turkish cup
col : MAX(winners_c_from_previous_round)
row 1: 54
*/"""]
    claims = ["what is the highest number of winner from a previous round?"]
    # inds from test split
    examples_prompt = PromptTemplate(input_variables=["SQL", "table", "claim", "thought", "output"], template=
    """
SQL Excuted: 
```{SQL}```
Sub-table: {table}

Query: {claim}
Thought: {thought}
Answer: {output}
    """)
    examples_dict = dict(zip(["SQL", "table", "claim", "thought", "output"], [sqls[0], tables_pipe[0], claims[0], thoughts[0], '54']))
    prompt_template = FewShotPromptTemplate(
        examples=[examples_dict],
        example_prompt=examples_prompt,
        prefix="""Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Think step by step and answer the question given in the query.
You should output in the following format:
Thought: your step by step thought
Answer: Only return the concise string instead of other format information. Do not repeat the question.
Below is an example.""",
        suffix=
        """
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Extra information:
{information}

Query: {query}""",
        input_variables=["table", "query", "SQL", "information"],
)
    return prompt_template



# def get_k_shot_with_answer(k: int=1):
#     sqls = ["SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';"
#             ]
#     thoughts = ["Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. 3 is the fewest points they received. "]
#     tables = ["<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>"]
#     claims = ["was 2 be the fewest point that roger dutton / tony wright receive?"]
#     # inds from test split
#     examples_prompt = PromptTemplate(input_variables=["SQL", "table", "claim", "thought", "output"], template=
#     """
# SQL Excuted: 
# ```{SQL}```
# Sub-table: {table}

# Query: {claim}
# Thought: {thought}
# Answer: {output}
#     """)
#     examples_dict = dict(zip(["SQL", "table", "claim", "thought", "output"], [sqls[0], tables[0], claims[0], thoughts[0], '3']))
#     prompt_template = FewShotPromptTemplate(
#         examples=[examples_dict],
#         example_prompt=examples_prompt,
#         prefix="""Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Think step by step and answer the question given in the query.
# You should output in the following format:
# Thought: your step by step thought
# Answer: Only return the concise string instead of other format information. Do not repeat the question.
# Below is an example.""",
#         suffix=
#         """
# SQL Excuted: 
# ```{SQL}```
# Sub-table: {table}
# Extra information:
# {information}

# Query: {query}""",
#         input_variables=["table", "query", "SQL", "information"],
# )
#     return prompt_template

In [9]:
def get_k_shot_with_aug(k: int=2):
    table_loader = TableLoader(table_name='wikitable', split='train', use_sample=True, small_test=False)
    examples_dict = []
    # examples_dict.extend([{"table": '<table>\n<caption>Hoot Kloot</caption>\n<thead>\n<tr><th> Number</th><th> Title</th><th> Directed_by_</th><th> Released_</th></tr>\n</thead>\n<tbody>\n<tr><td>1  </td><td>"Kloot\'s Kounty"           </td><td>Hawley Pratt  </td><td>1973       </td></tr>\n<tr><td>2  </td><td>"Apache on the County Seat"</td><td>Hawley Pratt  </td><td>1973       </td></tr>\n<tr><td>6  </td><td>"Stirrups and Hiccups"     </td><td>Gerry Chiniquy</td><td>1973       </td></tr>\n</tbody>\n</table>',
    #                                     "claim": table_loader.dataset[95]['question'],
    #                                     "linking": "the last title -> Released_, the last title-> Number, title -> Title, sid marcus -> Directed_by_",
    #                                     "output": "Title, Released_, Number, Directed_by_"}])
    examples_dict.extend([{"table": """/*
table caption : Hoot Kloot
col : Number | Title | Directed_by_ | Released_
row 1 : 1 | "Kloot's Kounty" | Hawley Pratt | 1973
row 2 : 2 | "Apache on the County Seat" | Hawley Pratt | 1973
row 6 : 6 | "Stirrups and Hiccups" | Gerry Chiniquy | 1973
*/""",
                                        "claim": table_loader.dataset[95]['question'],
                                        "linking": "the last title -> Released_, the last title-> Number, title -> Title, sid marcus -> Directed_by_",
                                        "output": "Title, Released_, Number, Directed_by_"}])
    examples_prompt = PromptTemplate(input_variables=["table", "claim", "output", "linking"], template=
    """
    Table: {table}
    Query: {claim}
    Column linking: {linking}
    Columns: {output}""")
    prompt_template = FewShotPromptTemplate(
        examples=examples_dict,
        example_prompt=examples_prompt,
        prefix=
        """
    Based on the Table below, your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the Table.
    Approach this task as follows:
    Read the query thoroughly and list every possible link from query term to column in the Table. 
    Then Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.""",
    # You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
    # Given the following table and query, you should output columns related to the query or contain useful information about the query. 
    # Here are some examples:""",
        suffix=
        """
    Table: {table}
    Extra information: {aug}
    
    Query: {claim}""",
        input_variables=["table", "claim", "aug"],
)
    return prompt_template


In [10]:
# import os
# from langchain_openai import AzureChatOpenAI
# from utils import parse_output
# table_loader = TableLoader(table_name='wikitable', split='train', use_sample=True, small_test=False)
# summary_information = pd.read_csv(f"../result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
# schema_information = pd.read_csv(f"../result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
# composition_information = pd.read_csv(f"../result/aug/{task_name}_{split}_composition.csv", index_col='table_id')


# sample = table_loader.normalize_table(table_loader.dataset[110])

# # summary_aug, column_aug = summary_information.loc[sample['table']['id']]['summary'], summary_information.loc[sample['table']['id']]['column_description'] 
# # col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
# # extra_col_info = []
# # for i_c in range(len(col_names)):
# #     extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
# formatter = TableFormat(format='none', data=sample)



# llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_aug(), verbose=True)
# batch_pred = llm_chain.batch([{
#     "claim": sample['query'],
#     'aug':  '',
#     # "query": "how many of the seasons games were played in the gold coast convention centre?",
#                             "table": TableFormat.format_PIPE(data= formatter.get_sample_data(sample_type='random',k=5, query=sample['query'],),table_caption=sample['table']['caption'])}],)
# print(batch_pred[0]['text'])
#     #                             "table": """table caption : 2008 - 09 nbl season. 

In [29]:
from utils import parse_output
def scene_A(query, sample, k =3, verbose=True):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""Our ultimate goal is to answer query based on the original table. Now we have a sub-table with rows randomly sampled from the original table, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Extra information: {aug}

Query: {claim}
SQL: """)
    formatter = TableFormat(format='none', data=sample, save_embedding=True, embeddings=embeddings)
    k_shot_prompt = get_k_shot_with_aug()
    formatter.normalize_schema(schema_information.loc[sample['table']['id']]['schema'])
    if k == 0:
        sample_data = formatter.get_sample_data(sample_type='head', k=k)
    else:
        sample_data = formatter.get_sample_data(sample_type='embedding', query=query, k=k)
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=k_shot_prompt, verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['table']['id']]['summary'], aug_information.loc[sample['table']['id']]['column_description'] 
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        
        stage_1_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_PIPE(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        if verbose:
            print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        extra_cols = formatter.get_sample_column(embeddings, column_aug)
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        # columns = list(set([c.strip() for c in stage_1_batch_pred.split(',')] + extra_cols))
        
        try: 
            # formatter.all_data = formatter.all_data.loc[:, columns]
            sample_data = add_row_number(sample_data.loc[:, columns])
        except:
            sample_data = add_row_number(sample_data)
        extra_information = []
        tuples = parse_specific_composition_zh(composition_information.loc[sample['table']['id']]['composition'], sample_data.columns)
        for col, com in tuples:
            if len(pd.unique(formatter.all_data[col])) < 6:
                com += f' (Values like {", ".join(list(pd.unique(formatter.all_data[col].astype(str))))})'
                extra_information.append(col + ':' + com)
            else:
                com += f' (Values like {", ".join(list(pd.unique(formatter.all_data[col][:3].astype(str))))}...)'
                extra_information.append(col + ':' + com)
        extra_information.append('row_number: row number in the original table')
        stage_2_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data = sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\nColumn information:\n' + '\n'.join(extra_information)
                                            })], return_only_outputs=True)[0]['text'].replace("–", "-").replace("—", "-").replace("―", "-").replace("−", "-")
        if verbose:
            print(stage_2_batch_pred)
    # stage 3: SQL Excution
    try: 
        execute_data = manager.execute_from_df(stage_2_batch_pred, add_row_number(formatter.all_data), table_name='DF')
    except:
        execute_data = formatter.all_data
        stage_2_batch_pred = 'SELECT * from DF;'
    if len(execute_data) == 0:
        return query, stage_2_batch_pred, 'No data from database', cb.total_tokens
    return query, stage_2_batch_pred, TableFormat.format_PIPE(data=execute_data), cb.total_tokens
    # return query, stage_2_batch_pred, execute_data, cb.total_tokens

In [12]:
import concurrent.futures
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)

### RUN

In [13]:
def eval_blury_string(pred_list):
    pred_label = []
    for pred in pred_list:
        predict_ans = pred.split('\n')[-1]
        if '0' in predict_ans:
            predict_ans = '0'
        elif '1' in predict_ans:
            predict_ans = '1'
        else:
            predict_ans = '2'
        pred_label.append(predict_ans)
    return pred_label

In [14]:
import concurrent.futures
from typing import List
import os
import json
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)

def save_csv(input_list: List[List], label_list: List, file_path):
    import pandas as pd
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    assert len(input_list) == len(label_list)
    df = pd.DataFrame()
    for i in range(len(label_list)):
        df[label_list[i]] = pd.Series(input_list[i])
    if os.path.exists(file_path) and file_path.endswith('.csv'):
        df_origin = pd.read_csv(file_path)
        df = pd.concat([df_origin, df], axis=0)
    df.to_csv(file_path, index=False, encoding='utf-8')

## 调整extrainformation的位置

In [15]:
from utils import parse_output
answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
                                    template="""
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering and answer the query using the final sub-table. 
SQL Excuted: 
```{SQL}```
Sub-table: 
{table}
Query: {claim}
Please provide a clear, complete statement in response to the query. If you cannot answer the query based on the sub-table, just say 'Cannot get answer from sub-table'.
""" )
def scene_B(query, sample, k=3, verbose=False):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""Our ultimate goal is to answer query based on the original table. Now we have a sub-table with rows sampled from the original table, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Extra information: {aug}

Query: {claim}
SQL: """)
    
    formatter = TableFormat(format='none', data=sample, save_embedding=True, embeddings=embeddings)
    formatter.normalize_schema(schema_information.loc[sample['table']['id']]['schema'])
    if k == 0:
        sample_data = formatter.get_sample_data(sample_type='head', k=k)
    else:
        sample_data = formatter.get_sample_data(sample_type='embedding', query=query, k=k)
    # get columns
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_aug(), verbose=verbose)
        summary_aug, column_aug = aug_information.loc[sample['table']['id']]['summary'], aug_information.loc[sample['table']['id']]['column_description'] 
        col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        extra_col_info = []
        for i_c in range(len(col_names)):
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        stage_1_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_PIPE(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n'.join(extra_col_info)
                                            })], return_only_outputs=True)[0]['text']
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        extra_cols = formatter.get_sample_column(embeddings, column_aug)
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = list(set([c.strip() for c in stage_1_batch_pred.split(',')] + extra_cols))
        try: 
            sample_data = add_row_number(sample_data.loc[:, columns])
        except:
            sample_data = add_row_number(sample_data)
        extra_information = []
        tuples = parse_specific_composition_zh(composition_information.loc[sample['table']['id']]['composition'], sample_data.columns)
        for col, com in tuples:
            if len(pd.unique(formatter.all_data[col])) < 6:
                com += f' (Values like {", ".join(list(pd.unique(formatter.all_data[col].astype(str))))})'
                extra_information.append(col + ':' + com)
            else:
                com += f' (Values like {", ".join(list(pd.unique(formatter.all_data[col][:3].astype(str))))}...)'
                extra_information.append(col + ':' + com)
        #  sample augmentation
        # extra_information = (parse_specific_composition(composition_information.loc[sample['table']['id']]['composition'], sample_data.columns))
        extra_information.append('row_number: row number in the table')
        stage_2_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_PIPE(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            'aug':  summary_aug + '\n Column information:' + '\n'.join(extra_information)
                                            })], return_only_outputs=True)[0]['text'].replace("–", "-").replace("—", "-").replace("―", "-").replace("−", "-")
    
        
        # stage 3: SQL Excution
        try: 
            execute_data= manager.execute_from_df(stage_2_batch_pred, add_row_number(formatter.all_data), table_name='DF')
        except:
            execute_data = formatter.all_data
            stage_2_batch_pred = 'SELECT * from DF;'
        llm_chain = LLMChain(llm=model, prompt=answer_instruction, verbose=verbose)
        response = llm_chain.batch([dict({'table': TableFormat.format_PIPE(execute_data),
                                                'claim': query,
                                                'SQL':  stage_2_batch_pred
                                                })], return_only_outputs=True)[0]['text']
    # print("total_tokens:", cb.total_tokens)
    return response, cb.total_tokens

In [16]:
from langchain_openai import AzureChatOpenAI
import os

os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0.01
)


In [32]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
from FlagEmbedding import FlagReranker
from openai import BadRequestError
from tqdm.notebook import tqdm
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=False, small_test=False)
# model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.com.cn/v1",
#                        openai_api_key="sk-bLZSHx4pKfPRZkYyIyyvUHSEjrlqj5sh2QIsxOM23yJnyoGD", temperature=0.01)
# save_path = f"../result/answer/wikitable_PIPE_{datetime.datetime.now().strftime('%m-%d_%H-%M-%S')}.csv"
save_path = f"../result/answer/wikitable_PIPE_05-28_01-34-01.csv"

# reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True)

muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
# template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
template = """
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task with the help of extra information below.

SQL Excuted: 
```{SQL}```
Sub-table:
{table}
Extra information:
{information}

Query: {query}
Think step by step and answer the last question given in the query. Only return the string instead of other format information. Do not repeat the question.
""" )
sample_k = 3
# Task: answer the last question given in the query. Only return the string instead of other format information. Do not repeat the question.
# Task: verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.


# muilti_answer_instruction = get_k_shot_with_answer()
# for sample_n in range(3):
tokens = []
outputs = []
labels = []
ids = []
extra_quries = []
i = 1142
# with tqdm(total=len(table_loader.dataset) , desc=f"Processing",ncols=1500) as pbar:
#     while i < len(table_loader.dataset):
#         try:
sample = table_loader.normalize_table(
                    table_loader.dataset[i])
all_tokens = 0
all_queries = []
formatter = TableFormat(format='none', data=sample, save_embedding=False)
sample_data = formatter.get_sample_data(sample_type='random', k=sample_k, query=sample['query'])
# with get_openai_callback() as cb:
#     llm_chain = LLMChain(llm=model, prompt=step_back_prompt_wiki, verbose=False)
#     batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_PIPE(sample_data)}], return_only_outputs=True)
#     all_queries.append(batch_pred[0]['text'].strip())
#     # llm_chain = LLMChain(llm=model, prompt=disambiguous_prompt_wiki, verbose=False)
#     # batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_PIPE(sample_data)}], return_only_outputs=True)
#     # all_queries.append(batch_pred[0]['text'].strip())
#     llm_chain = LLMChain(llm=model, prompt=decompose_prompt_wiki, verbose=False)
#     batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_PIPE(sample_data)}], return_only_outputs=True)
#     all_queries.extend([q.strip() for q in batch_pred[0]['text'].split(';')])
#     # print(all_queries)
# all_tokens += cb.total_tokens
# all_queries = list(set(all_queries))
# args_list = [{"query": q, "sample": sample, "k": sample_k} for q in all_queries]
# # print(args_list)
# ans_from_B = parallel_run_kwargs(scene_B, args_list)
# results = [res[0] for res in ans_from_B if 'Cannot get answer from sub-table' not in res[0] ]
# all_tokens += sum([res[1] for res in ans_from_B])
#With answer
results= []
with get_openai_callback() as cb:
    imp_input = scene_A(sample['query'], sample, sample_k, True)
    llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_answer(), verbose=True)
    batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(results)}], return_only_outputs=True)
# print(batch_pred[0])
all_tokens += cb.total_tokens
# print('ALL TOKENS', all_tokens)
ids.append(sample['id'])
labels.append(sample['query'])
outputs.append(batch_pred[0]['text'])
tokens.append(all_tokens)
extra_quries.append(';'.join(all_queries))
        #     if (i + 1) % 10 == 0:
        #             print(f'saving {i}')
        #             save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)
        #             outputs = []
        #             labels = []
        #             ids = []
        #             tokens = []
        #             extra_quries = []
        #     i += 1
        #     print(f' Process {i}')
        #     pbar.update(1)
        # except ValueError as e:
        #     print(f'******************Value Error {i}****************************')
        #     i += 1
        #     pbar.update(1)
        # except BadRequestError as e:
        #     print('*************************Bad Request**************')
        #     i += 1
        #     pbar.update(1)

    





> Entering new LLMChain chain...
Prompt after formatting:

    Based on the Table below, your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the Table.
    Approach this task as follows:
    Read the query thoroughly and list every possible link from query term to column in the Table. 
    Then Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.


    Table: /*
table caption : Hoot Kloot
col : Number | Title | Directed_by_ | Released_
row 1 : 1 | "Kloot's Kounty" | Hawley Pratt | 1973
row 2 : 2 | "Apache on the County Seat" | Hawley Pratt | 1973
row 6 : 6 | "Stirrups and Hiccups" | Gerry Chiniquy | 1973
*/
    Query: what was the last title that sid marcus directed?
    Column linking: the last title -> Released_, the last title-> Number, title ->

In [31]:
outputs

['Thought: Based on the sub-table provided, I need to look at the "Result" column to find the largest margin of victory for the Rams during the season. The largest margin of victory is indicated by the highest number in the "Result" column after the "W" (win) and before the "-" (score separator).\n\nAnswer: 35']

In [20]:
all_queries

['what were the results of each game?',
 'what was the largest margin of victory for the Rams?',
 "what was the rams' largest margin of victory during the season?"]

In [21]:
ans_from_B 

[('The results of each game were as follows:\n- W 20-17 (OT)\n- W 30-26\n- W 42-10\n- W 35-0\n- W 15-14\n- W 34-14\n- L 34-31\n- Bye\n- W 48-14\n- W 24-17\n- L 24-17\n- W 35-6\n- W 27-14\n- W 34-21\n- W 38-32\n- W 42-17\n- W 31-13',
  1162),
 ('The largest margin of victory for the Rams was 35 points, achieved in their game against the Detroit Lions on October 8, 2001, with a final score of 35-0.',
  1775),
 ("The Rams' largest margin of victory during the season was 44 points.",
  1076)]

In [22]:
a

NameError: name 'a' is not defined

In [ ]:
##### add residual 

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
# from FlagEmbedding import FlagReranker
from openai import BadRequestError, RateLimitError
from tqdm.notebook import tqdm
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=False, small_test=False)
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.tech/v1",
                       openai_api_key="sk-bLZSHx4pKfPRZkYyIyyvUHSEjrlqj5sh2QIsxOM23yJnyoGD", temperature=0.01)
save_path = f"../result/answer/wikitable_PIPE_05-28_01-34-01.csv"
sample_k = 3
data = pd.read_csv(save_path)

tokens = []
outputs = []
labels = []
ids = []
extra_quries = []
i = 0
with tqdm(total=len(table_loader.dataset), desc=f"Processing",ncols=150) as pbar:
    while i < len(table_loader.dataset):
        if table_loader.dataset[i]['id'] in list(data['ids']):
            i += 1
        else:
            try:
                sample = table_loader.normalize_table(
                                    table_loader.dataset[i])
                all_tokens = 0
                all_queries = []
                formatter = TableFormat(format='none', data=sample, save_embedding=False)
                sample_data = formatter.get_sample_data(sample_type='random', k=sample_k, query=sample['query'])
                with get_openai_callback() as cb:
                    llm_chain = LLMChain(llm=model, prompt=step_back_prompt_wiki, verbose=False)
                    batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_PIPE(sample_data)}], return_only_outputs=True)
                    all_queries.append(batch_pred[0]['text'].strip())
                    # llm_chain = LLMChain(llm=model, prompt=disambiguous_prompt_wiki, verbose=False)
                    # batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_PIPE(sample_data)}], return_only_outputs=True)
                    # all_queries.append(batch_pred[0]['text'].strip())
                    llm_chain = LLMChain(llm=model, prompt=decompose_prompt_wiki, verbose=False)
                    batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_PIPE(sample_data)}], return_only_outputs=True)
                    all_queries.extend([q.strip() for q in batch_pred[0]['text'].split(';')])
                    # print(all_queries)
                all_tokens += cb.total_tokens
                all_queries = list(set(all_queries))
                args_list = [{"query": q, "sample": sample, "k": sample_k} for q in all_queries]
                # print(args_list)
                ans_from_B = parallel_run_kwargs(scene_B, args_list)
                results = [res[0] for res in ans_from_B if 'Cannot get answer from sub-table' not in res[0] ]
                all_tokens += sum([res[1] for res in ans_from_B])
                #With answer
                # results= []
                with get_openai_callback() as cb:
                    imp_input = scene_A(sample['query'], sample, sample_k, False)
                    llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_answer(), verbose=False)
                    batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(results)}], return_only_outputs=True)
                # print(batch_pred[0])
                all_tokens += cb.total_tokens
                # print('ALL TOKENS', all_tokens)
                ids.append(sample['id'])
                labels.append(sample['query'])
                outputs.append(batch_pred[0]['text'])
                tokens.append(all_tokens)
                extra_quries.append(';'.join(all_queries))
                i += 1
                print(f'process {i}')
            except RateLimitError as e:
                print('*************************Rate limit**************')
                pass
        pbar.update(1)
save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)
    



Processing:   0%|                                                                                             …

process 4343
process 4344


In [ ]:
save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)

In [ ]:
len(outputs)

2

In [ ]:
save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)

In [ ]:
def parse_extra_info(summary_aug, column_aug, composition_aug, columns):
    col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
    items, crackets = parse_output(composition_aug, pattern = r'\d. (.+?): (.+)')
    assert len(items) == len(col_names)
    extra_col_info = []
    for i_c in range(len(col_names)):
        if col_names[i_c] in columns:
            extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]} {crackets[i_c]}')
            
    extra_col_info.append('row_number: row number in the original table')
    return summary_aug + '\n'.join(extra_col_info)